In [1]:
(require srfi/1)
(require plot/no-gui)

# Constants

In [2]:
(define prefound-primes '(1009 1013 1019
                          10007 10009 10037
                          100003 100019 100043
                          1000003 1000033 1000037)) ; abridged for testing
                          

In [3]:
(define prefound-primes '(1009 1013 1019
                          10007 10009 10037
                          100003 100019 100043
                          1000003 1000033 1000037
                          1000000007 1000000009 1000000021
                          10000000019 10000000033 10000000061
                          100000000003 100000000019 100000000057
                          1000000000039 1000000000061 1000000000063))

In [4]:
(define timestoavg 100000)

In [5]:
(define fermat-times 10)

In [6]:
(define (square x)
  (* x x))

(define (divides? a b)
  (= (remainder b a) 0))

(define (prime? n)
  (= n (smallest-divisor n)))

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n)
         n)
        ((divides? test-divisor n)
         test-divisor)
        (#t (find-divisor
               n
               (advancemethod test-divisor)))))

# 1-21
> Use the smallest-divisor procedure to find the smallest divisor of each of the following numbers: 199, 1999, 19999.

In [7]:
(define (smallest-divisor n)
  (find-divisor n 2)) ; stock method
  ;(find-divisor-integrated n 2)) ; hopefully faster

In [8]:
(define (prime-method x)
  (prime? x))
  ;(fast-prime? x fermat-times))

In [9]:
(define (advancemethod x)
  ;(next x)) ; 1-23 and forward
  (+ x 1)) ; prior to 1-23

In [10]:
(list 199 (smallest-divisor 199) 1999 (smallest-divisor 1999) 19999 (smallest-divisor 19999))

'(199 199 1999 1999 19999 7)

# 1-22
> write a procedure search-for-primes that checks the primality
of consecutive odd integers in a specified range. Use your procedure to find
the three smallest primes larger than 1000; larger than 10,000; larger than
100,000; larger than 1,000,000.

Times were too inconsistent, so I made an averaged version.

In [11]:
(define (avg-timed-prime-test n)
  (newline)
  (display n)
  (avg-start-prime-test n (current-inexact-milliseconds) 0 timestoavg))

(define (avg-start-prime-test n start-time total-time iter)
  (if (prime-method n)
      (let* ((this-time (- (current-inexact-milliseconds)
                          start-time))
            (new-total-time (+ total-time this-time)))
        (if (> iter 0)
          (avg-start-prime-test n (current-inexact-milliseconds) new-total-time (- iter 1))
          (list n (avg-report-prime (* 1.0 (/ new-total-time timestoavg))))))
      #f))
(define (avg-report-prime elapsed-time)
  (display " *** ")
  (display elapsed-time)
  elapsed-time)

(define (avg-search-for-primes minimum goal)
  (define m (if (even? minimum)
                (+ minimum 1)
                (minimum)))
  (define answer (avg-search-for-primes-iter m '() goal))
  (newline)
  answer)
(define (avg-search-for-primes-iter n lst goal)
  (if (= goal 0)
      lst
      (let ((x (avg-timed-prime-test n)))
        (if (not (equal? x #f))
            (avg-search-for-primes-iter (+ n 2) (cons x lst) (- goal 1))
            (avg-search-for-primes-iter (+ n 2) lst goal)))))

In [12]:
(define prime-times-set (append-map (lambda n (avg-search-for-primes (car n) 3)) '(10 100 1000 10000 100000 1000000)))


11 *** 0.0003431689453125
13 *** 0.00024562255859375
15
17 *** 0.00027622802734375

101 *** 0.0005096337890625
103 *** 0.000672861328125
105
107 *** 0.00049347412109375

1001
1003
1005
1007
1009 *** 0.00123931396484375
1011
1013 *** 0.00123373046875
1015
1017
1019 *** 0.00122795654296875

10001
10003
10005
10007 *** 0.00366612548828125
10009 *** 0.00503373291015625
10011
10013
10015
10017
10019
10021
10023
10025
10027
10029
10031
10033
10035
10037 *** 0.0036471923828125

100001
100003 *** 0.011121337890625
100005
100007
100009
100011
100013
100015
100017
100019 *** 0.01129197021484375
100021
100023
100025
100027
100029
100031
100033
100035
100037
100039
100041
100043 *** 0.0111245556640625

1000001
1000003 *** 0.03478732421875
1000005
1000007
1000009
1000011
1000013
1000015
1000017
1000019
1000021
1000023
1000025
1000027
1000029
1000031
1000033 *** 0.0348555908203125
1000035
1000037 *** 0.0350156884765625


In [13]:
prime-times-set

'((17 0.00027622802734375) (13 0.00024562255859375) (11 0.0003431689453125) (107 0.00049347412109375) (103 0.000672861328125) (101 0.0005096337890625) (1019 0.00122795654296875) (1013 0.00123373046875) (1009 0.00123931396484375) (10037 0.0036471923828125) (10009 0.00503373291015625) (10007 0.00366612548828125) (100043 0.0111245556640625) (100019 0.01129197021484375) (100003 0.011121337890625) (1000037 0.0350156884765625) (1000033 0.0348555908203125) (1000003 0.03478732421875))

In [14]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (function (lambda (x) (* (sqrt x) 0.00002))
                           #:color 'blue #:style 'dot
                           #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

(plot-pict '(prog #<procedure:...e/no-gui/plot2d.rkt:144:8> 400) 400 400 400 0 '() #f #f '#(#(11 1000037) #(4894474295696533/73786976294838206464 630786566300673/18014398509481984)) #<procedure:...lot2d/plot-area.rkt:202:4> #<procedure:...lot2d/plot-area.rkt:713:4> '#(0 0 1))

It takes roughly $\sqrt{10}$ times longer to verify a prime 10 times higher.

# 1.23

define a procedure next that returns 3 if its input is equal
to 2 and otherwise returns its input plus 2. Modify the smallest-divisor
procedure to use `(next test-divisor)` instead of `(+ test-divisor 1)`.

In [ ]:
;; We'll be using prefound primes which is larger than the original problem definition,
;;   so let's update this dataset before changing methods
(define prime-times-set (map avg-timed-prime-test prefound-primes))
prime-times-set


1009 *** 0.0012561328125
1013 *** 0.0013040576171875
1019 *** 0.00126231201171875
10007 *** 0.00367068603515625
10009 *** 0.00364358154296875
10037 *** 0.003665185546875
100003 *** 0.01112237548828125
100019 *** 0.01117121826171875
100043 *** 0.01114485595703125
1000003 *** 0.03470440185546875
1000033 *** 0.03478986083984375
1000037 *** 0.0347492431640625
1000000007 *** 1.0998548974609375
1000000009 *** 1.0997917651367188
1000000021 *** 1.100276328125
10000000019 *** 3.478043122558594
10000000033 *** 3.4781062890625
10000000061 *** 3.4937154052734374
100000000003 *** 11.075387043457031
100000000019 *** 11.017280869140626
100000000057 *** 11.070458459472656
1000000000039 *** 35.021130493164065
1000000000061 *** 34.803940615234374
1000000000063

In [ ]:
(define (next n)
  (if (= n 2)
      3
      (+ n 2)))

In [ ]:
(define (advancemethod x)
  (next x)) ; 1-23 and forward
  ;(+ x 1)) ; prior to 1-23

In [ ]:
;; Using prefound primes now
(define prime-times-next (map avg-timed-prime-test prefound-primes))
prime-times-next

In [ ]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (lines prime-times-next #:style 'short-dash
                        #:label "time with next procedure"
                        #:color 'green)
                 (points prime-times-next)
                 ;(function (lambda (x) (* (sqrt x) 0.000015))
                 ;          #:color 'blue #:style 'dot
                 ;          #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

Comparing (1000000 0.0347) for the old algorithm with (1000000 0.02), it also runs at $n * \sqrt{10}$, but takes about 3/5th the time. The (next)
function should allow for skipping about half the iterations necessary, which
is why I'm surprised it doesn't run in 1/2 the time. I wonder if the repeated
conditionals involved slow it down enough to be noticeable. Doesn't seem
likely. I'm going to look at what others say about this.

*Later...*

I was wrong, it *was* the conditional. I'll try inlining the code and see what can be gained.

In [ ]:
(define (find-divisor-integrated n test-divisor)
  ; when first run, runs logic for test-divisor = 2, then proceeds to odd looping
  (define (fdi-iter test-divisor)
    (cond ((> (square test-divisor) n)
           n)
          ((divides? test-divisor n)
           test-divisor)
          (#t (fdi-iter
               (+ test-divisor 2)))))
  (if (divides? test-divisor n) ;; Assuming test-divisor is 2
      test-divisor
      (fdi-iter (+ test-divisor 1))))

In [ ]:
(define (smallest-divisor n)
  ;(find-divisor n 2)) ; stock method
  (find-divisor-integrated n 2)) ; hopefully faster

In [ ]:
(define prime-times-fdi (map avg-timed-prime-test prefound-primes))
prime-times-fdi

In [ ]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (lines prime-times-next #:style 'short-dash
                        #:label "time with next procedure"
                        #:color 'green)
                 (points prime-times-next)
                 (lines prime-times-fdi #:style 'short-dash
                        #:label "time with inlined function"
                        #:color 'blue)
                 (points prime-times-fdi)
                 ;(function (lambda (x) (* (sqrt x) 0.000015))
                 ;          #:color 'blue #:style 'dot
                 ;          #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

NOTE: In the .scm version, this speedup was more drastic, running at twice the speed of the previous generation. This one is less drastic and it's inconsistent whether the speedup shows at all per rendering.

# 1.24
Modify the `timed-prime-test procedure` of Exercise 1.22 to use
`fast-prime?` (the Fermat method), and test each of the 12 primes you found in
that exercise. Since the Fermat test has $Θ(\log{n})$ growth, how would you expect
the time to test primes near 1,000,000 to compare with the time needed to
test primes near 1000? Do your data bear this out? Can you explain any
discrepancy you find?

In [ ]:
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp)
         (remainder
          (square (expmod base (/ exp 2) m))
          m))
        (else
         (remainder
          (* base (expmod base (- exp 1) m))
          m))))
(define (fermat-test n)
  (define (try-it a)
    (= (expmod a n n) a))
  (try-it (+ 1 (random (- n 1)))))
(define (fast-prime? n times)
  (cond ((= times 0) #t)
        ((fermat-test n)
         (fast-prime? n (- times 1)))
        (else #f)))

In [ ]:
(define (prime-method x)
  ;(prime? x))
  (fast-prime? x fermat-times))

In [ ]:
(define prime-times-fermat (map avg-timed-prime-test prefound-primes))
prime-times-fermat

In [ ]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (lines prime-times-next #:style 'short-dash
                        #:label "time with next procedure"
                        #:color 'green)
                 (points prime-times-next)
                 (lines prime-times-fdi #:style 'short-dash
                        #:label "time with inlined function"
                        #:color 'blue)
                 (points prime-times-fdi)
                 (lines prime-times-fermat #:style 'short-dash
                        #:label "time for fermat test"
                        #:color 'cyan)
                 ;(function (lambda (x) (* (sqrt x) 0.000015))
                 ;          #:color 'blue #:style 'dot
                 ;          #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))